## Housekeeping

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Source
from IPython.display import Image
import pydotplus
import math

## Preamble - Data Cleaning

To clean our data, we have reused some of the code we had employed earlier on in the course in our first assignment. We first concatenate these files in the below cell block, and run this file through our scrubber which is where we handle our data cleaning. For further details, you can review the scrubber notebook: Data_Scrubber.ipynb.

In [ ]:
#Combining the two datasets and saving as a new csv
df1 = pd.read_csv('covid19-cdc-22207000.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)
df2 = pd.read_csv('covid19-cdc-12468998.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

combined_df = pd.concat([df1, df2])

combined_df.to_csv('covid19-cdc-combined-22207000-12468998.csv', index=False)

print(len(combined_df))

The out of our data scrubber is <b>finalised-covid19-cdc-combined-22207000-12468998.csv</b>. This is the file we will use to train/test our models.

In [ ]:
#Reading the post-scrubbed CSV and checking length
df = pd.read_csv('finalised-covid19-cdc-combined-22207000-12468998.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

print(len(df))

# Task 1: Data Understanding and Preparation

# 1.1 Train/Test Split

With the data cleaning taken care of, we can now conduct a train text split on the data to prepare it for future machine learning tasks. Once the data is split, we can plot the train frame and investigate the relationships between the features.

In [ ]:
#data for task 1
df = pd.read_csv('finalised-covid19-cdc-combined-22207000-12468998.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

#train test split on data
train, test = train_test_split(df, test_size = 0.3, random_state=42)

In [ ]:
df.duplicated().sum()

In [ ]:
train.shape

In [ ]:
test.shape

# 1.2 Mapping our Training Set

## Correlation Between Continuous Features

In [ ]:
fig, axs = plt.subplots(figsize=(20, 6), ncols=1, nrows=1)
fig.set_facecolor('#f5f5f5')

plt.scatter(train['case_positive_specimen_interval'], train['case_onset_interval'], c='#84dcc6')
corr = train['case_onset_interval'].corr(train['case_positive_specimen_interval'])
print('Correlation: ', corr)
print('Weak positive correlation between these two variables.')
plt.show()


## Continuous Features Plotted Against Death

In [ ]:
fig, axs = plt.subplots(figsize=(20, 6), ncols=1, nrows=1)
plt.scatter(train['death_yn'], train['case_positive_specimen_interval'], c='#84dcc6')
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(20, 6), ncols=1, nrows=1)
fig.set_facecolor('#f5f5f5')
plt.scatter(train['death_yn'], train['case_onset_interval'], c='#84dcc6')
plt.show()

## Health Related Categorical Features Plotted Against Death

Here we are looking discern patterns from our dataset by plotting deaths (yes/no) against a select number of categorical features. The features we are going to examine first are those that relate to the health profile of our patients:

1. When the patient contracted COVID.
2. What US state the patient was in when they were reported of having COVID.
3. What age group the patient belongs to.
4. Whether the patient knew if they were exposed to COVID.
5. Whether or not the patient was hospitalised or not.
6. Whether or not the patient was taken to ICU.
7. And whether or not the patient exhibited underlying conditions which could exacerbate COVID.

In [ ]:
target_feature = train['death_yn']
select_features = [train['case_month'], train['res_state'], train['age_group'], train['hosp_yn'], train['exposure_yn'], train['icu_yn'], train['underlying_conditions_yn']]

colours = ['#84dcc6', '#ffc09f']

for feature in select_features:
  
    fig, axs = plt.subplots(ncols=2, figsize=(20, 8))
    train.groupby([feature, target_feature]).size().unstack().plot(kind='bar', stacked=True, ax=axs[0], color=colours)
    axs[0].set_xticklabels(axs[0].get_xticklabels(), rotation=90, fontsize=10)
    axs[0].set_ylabel('#')

    train.groupby([feature, target_feature]).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar', stacked=True, ax=axs[1], color=colours)
    axs[1].set_xticklabels(axs[1].get_xticklabels(), rotation=90, fontsize=10)
    axs[1].set_ylabel('%')
    
    fig.set_facecolor('#f5f5f5')
    plt.tight_layout()
    plt.show()

In [ ]:
target_feature = train['underlying_conditions_yn']
select_features = [train['hosp_yn'], train['icu_yn'], train['death_yn']]

colours = ['#84dcc6', '#ffc09f']

for feature in select_features:
  
    fig, axs = plt.subplots(ncols=2, figsize=(20, 8))
    train.groupby([feature, target_feature]).size().unstack().plot(kind='bar', stacked=True, ax=axs[0], color=colours)
    axs[0].set_xticklabels(axs[0].get_xticklabels(), rotation=90, fontsize=10)
    axs[0].set_ylabel('#')

    train.groupby([feature, target_feature]).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar', stacked=True, ax=axs[1], color=colours)
    axs[1].set_xticklabels(axs[1].get_xticklabels(), rotation=90, fontsize=10)
    axs[1].set_ylabel('%')
    
    fig.set_facecolor('#f5f5f5')
    plt.tight_layout()
    plt.show()

## Sociological Related Categorical Features Plotted Against Death

Here we are looking discern patterns from our dataset by plotting deaths (yes/no) against a number of sociological features: race, ethnicity, sex.

1. The race of the patient.
2. The ethnicity of the patient.
3. The sex of the patient.

In [ ]:

target_features = [train['race'], train['ethnicity'], train['sex']]
select_features = [train['death_yn']]

colours = ['#84dcc6', '#ffc09f']

for target_feature in target_features:

    for feature in select_features:

        fig, axs = plt.subplots(ncols=2, figsize=(20, 8))
        train.groupby([target_feature, feature]).size().unstack().plot(kind='bar', stacked=True, ax=axs[0], color=colours)
        axs[0].set_xticklabels(axs[0].get_xticklabels(), rotation=90, fontsize=10)
        axs[0].set_ylabel('#')

        train.groupby([target_feature, feature]).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar', stacked=True, ax=axs[1], color=colours)
        axs[1].set_xticklabels(axs[1].get_xticklabels(), rotation=90, fontsize=10)
        axs[1].set_ylabel('%')
        
        plt.tight_layout()
        plt.show()

## Observations:

1. As the COVID19 pandemic progressed, we can generally observe both a reduction in the number of deaths per month and the proporption of deaths with regards the whole, month-on-month. The outlier here is January 2022. From the CDC dataset provided, we cannot discern whether or not the patient was vaccinated, however we can use the months here as a proxy for the Vaccine rollout program in the US, which began mid-December 2020 and had more or less concluded by the end of the following year. The death ratio falling against total cases broadly follows the vaccination rollout scheme, despite this information not formally recorded in our dataset. Anecdotally, as COVID19 pandemic persisted more and more COVID mutations occured, often leading the new dominant COVID virus to be more transmissible at the expense of its lethality.

2. The data also suggests that age is a predominant and robust factor when gauging the impact contracting COVID will have on your health. Our data shows that those who belong to the oldest age bracket (65+) and test positive for COVID are far more likely to die than those who are younger (compare 65+ against 0-17 years old). Our data also shows that those who are older are more exposed to the risk of COVID, in that the oldest age bucket leads in terms of hospitalisations, ICU admissions, and underlying conditions in comparison to the other age brackets. This aligns with what is now known about COVID-19, that older adults and those with underlying health conditions are considered to be at a higher risk of severe illness and death from COVID-19. 

3. The data also demonstrates that having underlying medical conditions increases the risk of hospitalisation, ICU admission, and death from COVID-19. And while we do not get access to the comorbidities affecting the patients in our dataset, those who reported as having underlying conditions such as diabtes, hypertension, obesity (etc.), confer a much higher risk of death from COVID-19 than others.

4. The data also shows that in the event of being admitted to hospital or the ICU, the success of making a full recovery is limited, and drasticly decreases the older you are. The data also proves the opposite whereby the younger you are the more resilient you are to COVID, leading to far fewer hospitalisation, admissions to ICU, and deaths. 

5. The data also shows that minorities, categorised in our race and ethnicity features (Black, Hispanic, Asian), are hospitalised, admitted to ICU, and die more regularly than their white counterparts pro rata. Without more granular information on our patient it is hard to say exactly what factors are driving these disparities: access to healthcare; wealth; etc.,

6. The data also demonstrates that there is a weak correlation between case_onset_intervals and case_positive_specimen_intervals, with the correlation coefficient is close to zero. This indicates that the timing of when a person experiences symptoms (case onset interval) and when they test positive for COVID-19 (case positive specimen interval) are not strongly associated with each other.

7. Our data also shows that there is a weak negative correlation between our continuous features, and when we map these continuous features against our target feature, there is no real discernible pattern that differents yes from no.


## Selected Features

Based on the above observations, the subset of features we are going to use to model the health profile of our patients are: 

i. <b>Age Group</b>, 
ii. <b>Case Month</b>, 
iii. <b>Hospitalisation Status</b>, 
iv. <b>ICU Status</b>, 
v. <b>Underlying Conditions</b>. 

Let's transform these features into new features with discrete values so we can use them in our models.

#### Case Month

In [ ]:
month_map = {'2020-01': 0, 
             '2020-02': 1, 
             '2020-03': 2, 
             '2020-04': 3,
             '2020-05': 4,
             '2020-06': 5,
             '2020-07': 6,
             '2020-08': 7,
             '2020-09': 8,
             '2020-10': 9,
             '2020-11': 10,
             '2020-12': 11,
             '2021-01': 12,
             '2021-02': 13,
             '2021-03': 14,
             '2021-04': 15,
             '2021-05': 16,
             '2021-06': 17,
             '2021-07': 18,
             '2021-08': 19,
             '2021-09': 20,
             '2021-10': 21,
             '2021-11': 22,
             '2021-12': 23,
             '2022-01': 24,
             '2022-02': 25,
             '2022-03': 26,
             '2022-04': 27,
             '2022-05': 28,
             '2022-06': 29,
             '2022-07': 30,
             '2022-08': 31,
             '2022-09': 32,
             '2022-10': 33,
             '2022-11': 34
             }

train['case_month'] = train['case_month'].map(month_map)
test['case_month'] = test['case_month'].map(month_map)

#### Age Group

In [ ]:
age_map = {'0 - 17 years': 0, 
           '18 to 49 years': 1, 
           '50 to 64 years': 2, 
           '65+ years': 3
           }

train['age_group'] = train['age_group'].map(age_map)
test['age_group'] = test['age_group'].map(age_map)

#### Hospitalisation 

In [ ]:
hosp_map = { 'Unknown': 0, 
            'Yes': 1, 
            }

train['hosp_yn'] = train['hosp_yn'].map(hosp_map)
test['hosp_yn'] = test['hosp_yn'].map(hosp_map)

#### ICU 

In [ ]:
icu_map = {'Unknown': 0, 
            'Yes': 1, 
            }

train['icu_yn'] = train['icu_yn'].map(icu_map)
test['icu_yn'] = test['icu_yn'].map(icu_map)

### Exposure

In [ ]:
exposure_map = {'Unknown': 0, 
            'Yes': 1, 
            }

train['exposure_yn'] = train['exposure_yn'].map(icu_map)
test['exposure_yn'] = test['exposure_yn'].map(icu_map)

#### Underlying Conditions

In [ ]:
underlying_map = {'Unknown': 0, 
                  'Yes': 1, 
                }

train['underlying_conditions_yn'] = train['underlying_conditions_yn'].map(underlying_map)
test['underlying_conditions_yn'] = test['underlying_conditions_yn'].map(underlying_map)

In [ ]:
# train['definite_underlying'] = (train['underlying_conditions_yn'] == 1).astype(int)
# train['uncertain_underlying'] = (train['underlying_conditions_yn'] == 0).astype(int)

# test['definite_underlying'] = (test['underlying_conditions_yn'] == 1).astype(int)
# test['uncertain_underlying'] = (test['underlying_conditions_yn'] == 0).astype(int)

#### Death

In [ ]:
death_map = {'No': 0,
             'Yes': 1,
             }

train['death_yn'] = train['death_yn'].map(death_map)
test['death_yn'] = test['death_yn'].map(death_map)

# Task 2: Predictive Modeling - Linear Regression

The aim of this task is to perform some basic predictive analysis on the data prepared in task one. To this end, a linear regression model will be created using the training data yielded by the train-test split from the previous task. The features to be analyzed in this section are:

i. <b>age</b>, 
ii. <b>case month</b>, 
iii. <b>hospilisation status</b>, 
iv. <b>ICU status</b>, 
v. <b>underlying conditions</b>. 

## 2.1 Training our Linear Regression Model

Here we train our linear regression model based on the features we selected above. We train this model against our target feature which is death.

In [ ]:
linear_regression_x_train = train[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
linear_regression_y_train = train['death_yn']

linear_regression_x_test = test[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
linear_regression_y_test = test['death_yn']

Sanity checking feature selection for model

In [ ]:
print(linear_regression_x_train.shape)
print(linear_regression_x_test.shape)

In [ ]:
lr = LinearRegression()
lr.fit(linear_regression_x_train, linear_regression_y_train)
linear_regression_y_pred = lr.predict(linear_regression_x_train)

## 2.2 Evaluating the Coefficients of our Linear Regression Model

Here we print the coefficients of each feature we used to train the model

In [ ]:
lr_coeff = pd.DataFrame({'Feature': linear_regression_x_train.columns, 'Coeff': lr.coef_})
lr_coeff.reset_index(drop=True, inplace=True)
lr_coeff

### Observations:

1. <b>Case Month</b>: The coefficient for Case Month is -0.006671, which indicates that there is a small negative correlation between the month of the reported case and the target variable. That is to say that as the month of the reported case increases, the value of the target variable is expected to decrease slightly. This makes sense as we can understand time here to be a proxy (of sorts) for vaccination and the decreasing lethality of COVID19 as it mutated over the course of 34 months.

2. <b>Age Group</b>: The coefficient for this feature is 0.198640, which indicates a positive correlation between the age group and the target variable. In other words, as the age group of the patient increases, the value of the target variable is expected to increase. This follows the trends we observe from our dataset where the older the patient the more likely they are of dying from COVID19.

3. <b>Hospitlisation Status</b>: This feature represents whether the patient was hospitalised for COVID19 or not. As we might expect, the coefficient for this feature is relatively high at 0.413565, which demonstrates a strong positive correlation between hospitalisation and the target variable, death. 

4. <b> ICU Status </b>: Represents whether the patient was admitted to the ICU or not. The coefficient for this feature is 0.147415, which indicates a relatively strong positive correlation between ICU admission and our target variable, but not as strong as hospitalisation. However, we can understand this feature as a subset of hospilisation, as 99.8% of our patients who were admitted to ICU were first hospitalised, which in part can help us understand why this value is weighted less than hospitalisation.

5. <b> Underlying Health Conditions </b>: The coefficient for this feature is confusing at the model infers a negative correlation to having underlying health conditions. This finding is counterintuitive, as individuals with underlying health conditions are often thought to be at higher risk for adverse outcomes such as hospitalisation or death from COVID-19. However, it is important to remember that the coefficient represents the association between the predictor variable and the target outcome after accounting for the effects of other predictor variables in the model. In this case, it is possible that other predictor variables, such as age or hospitalisation status, may be more strongly associated with the target outcome than underlying health condition, and that the effect of underlying health condition on the target outcome is actually negative when these other factors are taken into account. Therefore, the negative coefficient for "definite_underlying" suggests that the presence of a definite underlying health condition alone may not be a strong predictor of the target outcome in this model.

## 2.3.1 Target Value and Class Prediction

Here we print the predicted target feature value for the first 10 training examples, and the predicted class for the first 10 training examples.

In [ ]:
# predict target feature values for first 10 training examples
linear_regression_y_train_pred_10 = lr.predict(linear_regression_x_train[:10])
y_train_pred_df = pd.DataFrame({'Predicted target feature values': linear_regression_y_train_pred_10})

# threshold predicted target feature values to get predicted classes
linear_regression_y_train_pred_class_10 = (lr.predict(linear_regression_x_train[:10]) > 0.5).astype(int)
y_train_pred_df['Predicted class'] = linear_regression_y_train_pred_class_10

print(y_train_pred_df)

## 2.3.2 Classification Evaluation Measures on Training Set

Below we print the classification evaluation measures computed on the full training set.

In [ ]:
# evaluate model on full training set
linear_regression_y_train_pred = lr.predict(linear_regression_x_train)

linear_regression_y_train_pred_class = (linear_regression_y_train_pred > 0.5).astype(int)

linear_regression_accuracy_train = accuracy_score(linear_regression_y_train, linear_regression_y_train_pred_class)
linear_regression_precision_train = precision_score(linear_regression_y_train, linear_regression_y_train_pred_class)
linear_regression_recall_train = recall_score(linear_regression_y_train, linear_regression_y_train_pred_class)
linear_regression_f1_train = f1_score(linear_regression_y_train, linear_regression_y_train_pred_class)

linear_regression_confusion_train = confusion_matrix(linear_regression_y_train, linear_regression_y_train_pred_class)

linear_regression_mse_train = mean_squared_error(linear_regression_y_train, linear_regression_y_train_pred_class)
linear_regression_mae_train = mean_absolute_error(linear_regression_y_train, linear_regression_y_train_pred_class)
linear_regression_rmse_train = np.sqrt(linear_regression_mse_train)
linear_regression_r2_train = r2_score(linear_regression_y_train, linear_regression_y_train_pred_class)

print("\nEvaluation metrics on the training set:")
print()
print("Accuracy: {:.2f}".format(linear_regression_accuracy_train))
print("Precision: {:.2f}".format(linear_regression_precision_train))
print("Recall: {:.2f}".format(linear_regression_recall_train))
print("F1 score: {:.2f}".format(linear_regression_f1_train))
print()
print("Confusion matrix:\n", linear_regression_confusion_train)
print()
print("Mean Squared Error: {:.2f}".format(linear_regression_mse_train))
print("Mean Absolute Error: {:.2f}".format(linear_regression_mae_train))
print("R^2: {:.2f}".format(linear_regression_r2_train))
print("Root Mean Squared Error: {:.2f}".format(linear_regression_rmse_train))

Adding all of our performance metrics to an iterable so we can loop over them for graph generation later

In [ ]:
list_of_linear_regression_training_stats = [linear_regression_accuracy_train, linear_regression_precision_train, linear_regression_recall_train, linear_regression_mse_train, linear_regression_mae_train, linear_regression_rmse_train, linear_regression_r2_train]

## Observations:

Based on the provided evaluation metrics, the model seems to quite well on our training set. 

1. The <b>Accuracy</b> of 0.88 indicates that the model predicted correctly 88% of the time. 
2. <b>Precision</b> is a measure of how many of the instances that the model predicted as positive were actually positive. A precision score of 0.81 means that out of all the instances that the model predicted as positive, 87% of the time the instance was actually positive.
3. For <b>Recall</b> we measured quite poorly, all things considered. A recall score of 0.65 means that the model correctly identified 65% of all the actual positive instances.
4. The <b>F1 Score</b> of 0.75 is a harmonic mean of precision and recall, which is a useful metric when there is an imbalance in the number of instances of each class. In this case, the F1 score indicates a good balance between precision and recall.
5. An <b>MSE</b> of 0.12 represents the average squared difference between the predicted and actual outcome values. Our low MSE score here is indicative of decent performance.
6. The R^2 value of 0.40 indicates that 40% of the variation in the outcome variable can be explained by the predictor variables included in the model.
7. Our <b>Confusion Matrix</b> is a visualisation of the distribution of correct and incorrect predictions across the two classes in table format. In this case, the model made 17585 true negative predictions, 649 false negative predictions, 2251 false positive predictions, and 4258 true positive predictions.





<b>Finish commmentary here on R2 etc.....</b>

In [ ]:
print("Linear Regression Classification Report:\n", classification_report(linear_regression_y_train, linear_regression_y_train_pred_class))

## 2.4.1 Classification Evaluation Comparison Between Training and Testing Sets

In [ ]:
#Evaluate model on full test set
linear_regression_y_test_pred = lr.predict(linear_regression_x_test)

linear_regression_y_test_pred_class = (linear_regression_y_test_pred > 0.5).astype(int)

linear_regression_accuracy_test = accuracy_score(linear_regression_y_test, linear_regression_y_test_pred_class)
linear_regression_precision_test = precision_score(linear_regression_y_test, linear_regression_y_test_pred_class)
linear_regression_recall_test = recall_score(linear_regression_y_test, linear_regression_y_test_pred_class)
linear_regression_f1_test = f1_score(linear_regression_y_test, linear_regression_y_test_pred_class)

linear_regression_confusion_test = confusion_matrix(linear_regression_y_test, linear_regression_y_test_pred_class)

linear_regression_mae_test = mean_absolute_error(linear_regression_y_test, linear_regression_y_test_pred_class)
linear_regression_mse_test = mean_squared_error(linear_regression_y_test, linear_regression_y_test_pred_class)
linear_regression_rmse_test = np.sqrt(linear_regression_mse_test)
linear_regression_r2_test = r2_score(linear_regression_y_test, linear_regression_y_test_pred_class)

print("\nEvaluation metrics on the test set:")
print("Accuracy: {:.2f}".format(linear_regression_accuracy_test))
print("Precision: {:.2f}".format(linear_regression_precision_test))
print("Recall: {:.2f}".format(linear_regression_recall_test))
print("F1 score: {:.2f}".format(linear_regression_f1_test))
print()
print("Confusion matrix:\n", linear_regression_confusion_test)
print()
print("Mean Squared Error: {:.2f}".format(linear_regression_mse_test))
print("Mean Absolute Error: {:.2f}".format(linear_regression_mae_train))
print("R^2: {:.2f}".format(linear_regression_r2_test))
print("Root Mean Squared Error: {:.2f}".format(linear_regression_rmse_test))

list_of_linear_regression_testing_stats = [linear_regression_accuracy_test, linear_regression_precision_test, linear_regression_recall_test, linear_regression_mse_test, linear_regression_mae_test, linear_regression_rmse_test, linear_regression_r2_test]

In [ ]:
print("Classification Report:\n", classification_report(linear_regression_y_test, linear_regression_y_test_pred_class))

In [ ]:
metrics = ['Accuracy', 'Precision', 'Recall', 'MSE', 'MAE', 'RMSE', 'R2']

for i in range(len(metrics)):
    print(metrics[i])
    print('='*30)
    print('Trained Data: ', list_of_linear_regression_training_stats[i])
    print('Tested Data: ', list_of_linear_regression_testing_stats[i])
    print()

As might be expected, the model performance on the training data and test data was incredibly similar. Both confusion matrices performed well on predicting negatives, but performed slightly worse when detecting positives. Metrics like accuracy, presicion, F1, and MSE proved identical, with only minor differences in Recall and R^2 to the tune of .01. 

Overall, performance on both the testing and training data seems to indicate a model with generally high accuracy, and precision, reacall, and F1 scores in excess of 90%. This kind of performance bodes well for a linear regression model's potential as a best fit for this particular problem.

## 2.4.2 K-Fold Cross Validation

In [ ]:
#performing 5 fold K-fold cross validation on linear regression model using full dataset

#concatenating train and test datasets for independent and dependent variables
X = pd.concat([train[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']], 
                                 test[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]])
y = pd.concat([train['death_yn'], test['death_yn']]) 

#folds for cross validation
folds = 5

#creating K-Fold object
kf = KFold(n_splits=folds, shuffle=True, random_state=42)

#K-fold cross validation scores
mse_scores = -cross_val_score(lr, X, y, cv=folds, scoring="neg_mean_squared_error")

#printing results
print(f"MSE scores: {mse_scores}")

The MSE scores produced by the K-Fold cross valdation process performed on the linear regression model were generally very low, with none exceeding 0.083. This indicates that the linear regression implementation is relatively sucessful and predicting the dependent outcome based on the provided independent features, even when performed on subsets of the full dataset. This is consistent with the relatively high accuracy yielded by the model when performing analysis of the testing and training data. Taken together, the model performance information and the K-fold cross validation information give us a good indication that this model would likely perform well on new data.

# Task 3: Predictive Modeling - Logistic Regression

The aim of this task is to perform some basic predictive analysis on the data prepared in task one. To this end, a logistic regression model will be created using the training data yielded by the train-test split from the previous task. The features to be analyzed in this section are:

i. <b>age</b>, 
ii. <b>case month</b>, 
iii. <b>hospilisation status</b>, 
iv. <b>ICU status</b>, 
v. <b>underlying conditions</b>. 

## 3.1 Training our Logistic Regression Model

Here we train our logistic regression model based on the features we selected above. We train this model against our target feature which is death.

In [ ]:
logistic_regression_x_train = train[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
logistic_regression_y_train = train['death_yn']

logistic_regression_x_test = test[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
logistic_regression_y_test = test['death_yn']

In [ ]:
logr = LogisticRegression()
logr.fit(logistic_regression_x_train, logistic_regression_y_train)
logistic_regression_y_pred = logr.predict(logistic_regression_x_train)

Sanity checking feature selection for model

In [ ]:
print(logistic_regression_x_train.shape)
print(logistic_regression_x_test.shape)

## 3.2 Evaluating the Coefficients of our Logistic Regression Model

Here we print the coefficients of each feature we used to train the model

In [ ]:
logr_coeff = pd.DataFrame({'Feature': logistic_regression_x_train.columns, 'Coeff': logr.coef_.ravel()})
logr_coeff.reset_index(drop=True, inplace=True)
logr_coeff

## Observations:

1. Similar to our Linear Regression model, the negative coefficient of -0.084 indicates that there is an inverse relationship between <b>Case Month</b> and the positive class probability. That is to say that when the case month increases, the log-odds of the positive class decreases. This again complements our initial observations that case month is a fair proxy for baseline vaccination percentages across the american population and the decreasing lethality of the COVID19 pathogen.
2. Again similar to our Linear Regression Model, the Logistic Regression Model weights <b>Age Group</b> quite heavilty. The positive coefficient 2.588 indicates that as the age of our patient increases, the log-odds of the positive class also increase. This means there is a direct relationship between the age of our patient andthe positive class probability. Suffice it to say that older patients are more likely to die than younger patients of COVID.
3. The Logistic Regression Model also weights <b>Hospilisation Status</b> and <b>ICU Status</b> quite highly, with coefficients of 2.507 and 2.322 respectively. This infers that when a patient is hospitalised, or subsequently taken to the ICU after their hospitalisation, are more likely to die from COVID than those who have not rushed to hospital or been admitted to an ICU.
4. Again the cresults for <b>Underlying Conditions</b> seem to be rather counter-intuitive, as our Logistic Regression Model demonstrates that patients with underlying conditions are less likely to die than those without underlying conditions. This is reflected in the weak negative coefficient of -0.647. 





## 3.3.1 Target Value and Class Prediction

Here we print the predicted target feature value for the first 10 training examples, and the predicted class for the first 10 training examples.

In [ ]:
logistic_regression_y_train_pred_10 = logr.predict_proba(logistic_regression_x_train)[:, 1][:10]
logistic_regression_y_train_pred_class_10 = (logr.predict(logistic_regression_x_train[:10]) >= 0.5).astype(int)

logistic_regression_output_df = pd.DataFrame({
    "Predicted Values": logistic_regression_y_train_pred_10,
    "Predicted Class": logistic_regression_y_train_pred_class_10
})

print(logistic_regression_output_df)

## 3.3.2 Classification Evaluation Measures on Training Set

In [ ]:
# predict target feature values for full training set
logistic_regression_y_train_pred = logr.predict_proba(logistic_regression_x_train)[:, 1]

# threshold predicted target feature values at 0.5 to get predicted class for full training set
logistic_regression_y_train_pred_class = (logistic_regression_y_train_pred >= 0.5).astype(int)

logistic_regression_accuracy_train = accuracy_score(logistic_regression_y_train, logistic_regression_y_train_pred_class)
logistic_regression_confusion_train = confusion_matrix(logistic_regression_y_train, logistic_regression_y_train_pred_class)
logistic_regression_recall_train = recall_score(logistic_regression_y_train, logistic_regression_y_train_pred_class)
logistic_regression_f1_train = f1_score(logistic_regression_y_train, logistic_regression_y_train_pred_class)

logistic_regression_precision_train = precision_score(logistic_regression_y_train, logistic_regression_y_train_pred_class)

logistic_regression_mae_train = mean_absolute_error(logistic_regression_y_train, logistic_regression_y_train_pred)
logistic_regression_mse_train = mean_squared_error(logistic_regression_y_train, logistic_regression_y_train_pred)
logistic_regression_rmse_train = np.sqrt(logistic_regression_mse_train)
logistic_regression_r2_train = r2_score(logistic_regression_y_train, logistic_regression_y_train_pred)

print("\nEvaluation metrics on the training set:")
print()
print("Accuracy: {:.2f}".format(logistic_regression_accuracy_train))
print("Precision: {:.2f}".format(logistic_regression_precision_train))
print("Recall: {:.2f}".format(logistic_regression_recall_train))
print("F1 score: {:.2f}".format(logistic_regression_f1_train))
print()
print("Confusion matrix:\n", logistic_regression_confusion_train)
print()
print("Mean Squared Error: {:.2f}".format(logistic_regression_mse_train))
print("Mean Absolute Error: {:.2f}".format(logistic_regression_mae_train))
print("R2 score: {:.2f}".format(logistic_regression_r2_train))
print("Root Mean Squared Error: {:.2f}".format(logistic_regression_rmse_train))

Adding all of our performance metrics to an iterable so we can loop over them for graph generation later

In [ ]:
list_of_logistic_regression_training_stats = [logistic_regression_accuracy_train, logistic_regression_precision_train, logistic_regression_recall_train, logistic_regression_mse_train, logistic_regression_mae_train, logistic_regression_rmse_train, logistic_regression_r2_train]

## 3.4.1: Classification Evaluation Comparison Between Training and Testing Sets

In [ ]:
# Evaluate model on full test set
logistic_regression_y_test_pred = logr.predict_proba(logistic_regression_x_test)[:, 1]

# threshold predicted target feature values at 0.5 to get predicted class for full test set
logistic_regression_y_test_pred_class = (logistic_regression_y_test_pred >= 0.5).astype(int)

logistic_regression_accuracy_test = accuracy_score(logistic_regression_y_test, logistic_regression_y_test_pred_class)
logistic_regression_precision_test = precision_score(logistic_regression_y_test, logistic_regression_y_test_pred_class)
logistic_regression_recall_test = recall_score(logistic_regression_y_test, logistic_regression_y_test_pred_class)
logistic_regression_f1_test = f1_score(logistic_regression_y_test, logistic_regression_y_test_pred_class)

logistic_regression_confusion_test = confusion_matrix(logistic_regression_y_test, logistic_regression_y_test_pred_class)

logistic_regression_mae_test = mean_absolute_error(logistic_regression_y_test, logistic_regression_y_test_pred)
logistic_regression_mse_test = mean_squared_error(logistic_regression_y_test, logistic_regression_y_test_pred)
logistic_regression_rmse_test = np.sqrt(logistic_regression_mse_test)
logistic_regression_r2_test = r2_score(logistic_regression_y_test, logistic_regression_y_test_pred)

print("\nEvaluation metrics on the test set:")
print()
print("Accuracy: {:.2f}".format(logistic_regression_accuracy_test))
print("Precision: {:.2f}".format(logistic_regression_precision_test))
print("Recall: {:.2f}".format(logistic_regression_recall_test))
print("F1 score: {:.2f}".format(logistic_regression_f1_test))
print()
print("Confusion matrix:\n", logistic_regression_confusion_test)
print()
print("Mean Absolute Error: {:.2f}".format(logistic_regression_mae_test))
print("Mean Squared Error: {:.2f}".format(logistic_regression_mse_test))
print("Root Mean Squared Error: {:.2f}".format(logistic_regression_rmse_test))
print("R2 score: {:.2f}".format(logistic_regression_r2_test))

list_of_logistic_regression_testing_stats = [logistic_regression_accuracy_test, logistic_regression_precision_test, logistic_regression_recall_test, logistic_regression_mse_test, logistic_regression_mae_test, logistic_regression_rmse_test, logistic_regression_r2_test]

In [ ]:
print("Classification Report:\n", classification_report(logistic_regression_y_test, logistic_regression_y_test_pred_class))

In [ ]:
metrics = ['Accuracy', 'Precision', 'Recall', 'MSE', 'MAE', 'RMSE', 'R2']

for i in range(len(metrics)):
    print(metrics[i])
    print('='*30)
    print('Trained Data: ', list_of_logistic_regression_training_stats[i])
    print('Tested Data: ', list_of_logistic_regression_testing_stats[i])
    print()

Again, the performance of the logistic regression model between training and test data is quite similar. Accuracy and F1 score remain consistent, while precision and recall are again off by only .01. Performance on the confusion matrices are, again, very similar too. Overll, the metrics fromt he training and testing data indicate a well-performing model, at least on mpar with if not slightly superior to the linear regression model. This outcome might be expected with a logistic regression model, due to the binary nature of the dependent feature as well as several of the independent features.

In [ ]:
#performing 5 fold K-fold cross validation on logistic regression model using full dataset

#concatenating train and test datasets for independent and dependent variables
X = pd.concat([train[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']], 
                                 test[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]])
y = pd.concat([train['death_yn'], test['death_yn']]) 

#folds for cross validation
folds = 5

#creating K-Fold object
kf = KFold(n_splits=folds, shuffle=True, random_state=42)

#K-fold cross validation scores
mse_scores = -cross_val_score(logr, X, y, cv=folds, scoring="neg_mean_squared_error")

#printing results
print(f"MSE scores: {mse_scores}")

The MSE scores produced by the K-Fold cross validation process on the logistic regression model were again very low, with none exceeding 0.11. his indicates that the linear regression implementation is relatively sucessful and predicting the dependent outcome based on the provided independent features, even when performed on subsets of the full dataset. This is consistent with the relatively high accuracy yielded by the model when performing analysis of the testing and training data. Taken together, the model performance information and the K-fold cross validation information give us a good indication that this model would likely perform well on new data. 

One thing to note, however, is that the MSE scores produced were slightly lower than those of the linear regression model's cross validation data. This could indicate there the logistic regression model has some comparatively negative model performance, such as overfitting to certain parts of the data, or not performing as well on new data. Regardless, the accuracy remains high and the MSE scores remain low, indicating good model performance overall.

# Task 4: Predictive Modeling - Random Forest Classifier

The aim of this task is to perform some basic predictive analysis on the data prepared in task one. To achieve thi we are going to create a random forest classifer model. Again the features we are going to use to train this model are:

i. <b>age</b>, 
ii. <b>case month</b>, 
iii. <b>hospilisation status</b>, 
iv. <b>ICU status</b>, 
v. <b>underlying conditions</b>. 

 The model will then be evaluated based on its performance using the training data, the test data, and other linear regression models trained on the full dataset. 

## 4.1

Here we train our random forest classifier model based on the features we selected above. We train this model against our target feature which is death.

In [ ]:
random_forest_x_train = train[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
random_forest_y_train = train['death_yn']

random_forest_x_test = test[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
random_forest_y_test = test['death_yn']

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(random_forest_x_train, random_forest_y_train)
random_forest_y_pred = rfc.predict(random_forest_x_test)

## 4.2.1 How does Random Forest Work?

Random forest is an ensemble learning algorithm that creates multiple decision trees and then aggregates their results to make a final prediction. It works by creating a set of decision trees, where each tree is trained on a random subset of the data and a random subset of the features. During training, the algorithm chooses the best feature to split the data on, and then continues splitting until the data is fully separated, or a stopping criterion is met.

Once the decision trees are built, the algorithm aggregates their results to make a final prediction. As we are using a random forest classifier model, this is done by taking the mode of the predicted classes across all the trees.

In [ ]:
decision_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
decision_tree.fit(random_forest_x_train, random_forest_y_train)

dot_data = export_graphviz(decision_tree, 
                           out_file=None, 
                           feature_names=random_forest_x_train.columns, 
                           class_names=['0', '1'],
                           filled=True, 
                           rounded=True, 
                           special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data)

graph.set_size('"20,8!"')
Image(graph.create_png())

## 4.2.2 Evaluating the Feature Importance of our Random Forest Classifier Model


In [ ]:
feature_importance = pd.DataFrame({'Feature': random_forest_x_train.columns, 'Significance':rfc.feature_importances_})
feature_importance.sort_values('Significance', ascending=False)

### Observations:

1. <b>Age Group here</b> leads the way in terms of feature importance, such that it is the dominant feature in terms of its contribution to the prediction model. It has an importance value of 0.537, which is significantly higher than the second most important feature, <b>hospitalisation Status</b>, with an importance value of 0.302. This matches our previous analysis where we observed that those who belonged to the oldest age bracket were the most exposed group to COVID, and resulted in the highest amount deaths.
2. <b>Hospitalisation Status</b> is our second most important feature and from previous analysis we have observed the high correlation between hospitlisation and the target feature. Here our models weights this feature quite heavily, which follows our previous observations and analysis; those who were admitted to hospital for COVID more than likely had a more serious case of COVID19 than those who were not admited to hospital. 
3. <b>Case Month</b> is weighted more highly here than it has been in other models. This is quite interesting, as earlier we posited that case month could potentially serve as  a proxy for COVID virus lethality (given the continuous evolution and mutation of the dominant COVID pathogen) and general vaccination rates (US vaccine program began in December 2020). While this is somewhat supposition, it is interesting that our model has picked up on this feature as being more imporant than both <b>ICU Status</b> and <b>Underlying Conditions</b>
4. Intersting that both <b>ICU Status</b> and <b>Underlying Conditions</b> were weighted quite lowly in this model. Again, this seems counter-intuitive as these are both important factors in determining the severity of COVID-19 cases. However, it's important to note that feature importance in a machine learning model is relative to the other features included in the model. So, while ICU status and underlying conditions may be important factors in determining the severity of COVID-19 cases, they may not be the most important factors in relation to the other features included in the model. It's also possible that the data used to train the model did not have a strong enough relationship between ICU status/underlying conditions and the target variable (severity) for these features to have a high weight in the model.

## 4.3.1

Here we print the predicted target feature value for the first 10 training examples, and the predicted class for the first 10 training examples.

In [ ]:
random_forest_y_train_pred_10 = rfc.predict_proba(random_forest_x_train)[:, 1][:10]
random_forest_y_train_pred_class_10 = (rfc.predict(random_forest_x_train[:10]) > 0.5).astype(int)

random_forest_output_df = pd.DataFrame({
    "Predicted Values": random_forest_y_train_pred_10,
    "Predicted Class": random_forest_y_train_pred_class_10
})

print(random_forest_output_df)

## 4.3.2 Classification Evaluation Measures on Training Set

Here we print a few classification evaluation measures computed on the full training set

In [ ]:
# predict target feature values for full training set
random_forest_y_train_pred = rfc.predict_proba(random_forest_x_train)[:, 1]

# threshold predicted target feature values at 0.5 to get predicted class for full training set
random_forest_y_train_pred_class = (random_forest_y_train_pred >= 0.5).astype(int)

random_forest_accuracy_train = accuracy_score(random_forest_y_train, random_forest_y_train_pred_class)
random_forest_precision_train = precision_score(random_forest_y_train, random_forest_y_train_pred_class)
random_forest_recall_train = recall_score(random_forest_y_train, random_forest_y_train_pred_class)
random_forest_f1_train = f1_score(random_forest_y_train, random_forest_y_train_pred_class)

random_forest_confusion_train = confusion_matrix(random_forest_y_train, random_forest_y_train_pred_class)

random_forest_mae_train = mean_absolute_error(random_forest_y_train, random_forest_y_train_pred_class)
random_forest_mse_train = mean_squared_error(random_forest_y_train, random_forest_y_train_pred_class)
random_forest_rmse_train = np.sqrt(random_forest_mse_train)
random_forest_r2_train = r2_score(random_forest_y_train, random_forest_y_train_pred_class)

print("\nEvaluation metrics on the training set:")
print("Accuracy: {:.2f}".format(random_forest_accuracy_train))
print("Precision: {:.2f}".format(random_forest_precision_train))
print("Recall: {:.2f}".format(random_forest_recall_train))
print("F1 score: {:.2f}".format(random_forest_f1_train))
print()
print("Confusion matrix:\n", random_forest_confusion_train)
print()
print("Mean Squared Error: {:.2f}".format(random_forest_mse_train))
print("Mean Absolute Error: {:.2f}".format(random_forest_mae_train))
print("R2 score: {:.2f}".format(random_forest_r2_train))
print("Root Mean Squared Error: {:.2f}".format(random_forest_rmse_train))


Again, adding our performance metrics to an iterable so we can loop over them.

In [ ]:
list_of_random_forest_training_stats = [random_forest_accuracy_train, random_forest_precision_train, random_forest_recall_train, random_forest_mse_train, random_forest_mae_train, random_forest_rmse_train, random_forest_r2_train]

### Observations:

Based on the provided evaluation metrics, the model seems to perform well on the training set. 

1. The <b>accuracy</b> of 0.91 indicates that the model predicted correctly 91% of the time. 
2. <b>Precision</b> is a measure of how many of the instances that the model predicted as positive were actually positive. In other words, it measures how many true positives the model predicted out of all the instances it predicted as positive. A precision score of 0.81 means that out of all the instances that the model predicted as positive, only 81% of them were actually positive.
3. <b>Recall</b> is a measure of how many of the actual positive instances the model correctly identified. In other words, it measures how many true positives the model predicted out of all the actual positive instances. A recall score of 0.89 means that the model correctly identified 89% of all the actual positive instances.
4. The <b>F1 Score</b> of 0.84 is a harmonic mean of precision and recall, which is a useful metric when there is an imbalance in the number of instances of each class. In this case, the F1 score indicates a good balance between precision and recall.
5. Our <b>Confusion Matrix</b> is a visualisation of the distribution of correct and incorrect predictions across the two classes in table format. In this case, the model made 16843 true negative predictions, 1405 false negative predictions, 747 false positive predictions, and 5830 true positive predictions. The large number of true negatives and true positives indicates that the model is doing well in correctly identifying both negative and positive instances. However, the relatively high number of false negatives and false positives suggests that there is still room for improvement in the model. <b>In particular, we should pay attention to the False Negative value, as it is of paramount importance that we reduce the number of misdiagnoses for those who are vulnerable to COVID19</b>

In [ ]:
print("Random Forest Classifier Classification Report:\n", classification_report(random_forest_y_train, random_forest_y_train_pred_class))

From the classification report, we can see that our model is more comfortable in predicting the negative class with a precision of 0.96 and a recall of 0.92. This means that among all the instances that the model predicted as negative, 96% of them were actually negative and the model correctly identified 92% of all actual negative instances.

However, our random forest classifier  model is less comfortable in predicting the positive class (class 1) with a precision of 0.81 and a recall of 0.89. Only 81% of them were actually positive and the model correctly identified 89% of all actual positive instances. 

In comparison with the other models, despite the random forest classifier's higher overall accuracy, it has a lower accuracy than linear regression and logistic regression when attempting to predict the positive class. As this is our priority we should always bear this in mind when evalutating the performance of our models.

## 4.4.1: Classification Evaluation Comparison Between Training and Testing Sets

In [ ]:
# predict target feature values for full testing set
random_forest_y_test_pred = rfc.predict_proba(random_forest_x_test)[:, 1]

# threshold predicted target feature values at 0.5 to get predicted class for full testing set
random_forest_y_test_pred_class = (random_forest_y_test_pred >= 0.5).astype(int)

random_forest_accuracy_test = accuracy_score(random_forest_y_test, random_forest_y_test_pred_class)
random_forest_precision_test = precision_score(random_forest_y_test, random_forest_y_test_pred_class)
random_forest_recall_test = recall_score(random_forest_y_test, random_forest_y_test_pred_class)
random_forest_f1_test = f1_score(random_forest_y_test, random_forest_y_test_pred_class)

random_forest_confusion_test = confusion_matrix(random_forest_y_test, random_forest_y_test_pred_class)

random_forest_mae_test = mean_absolute_error(random_forest_y_test, random_forest_y_test_pred_class)
random_forest_mse_test = mean_squared_error(random_forest_y_test, random_forest_y_test_pred_class)
random_forest_rmse_test = np.sqrt(random_forest_mse_test)
random_forest_r2_test = r2_score(random_forest_y_test, random_forest_y_test_pred_class)

print("\nEvaluation metrics on the testing set:")
print()
print("Accuracy: {:.2f}".format(random_forest_accuracy_test))
print("Precision: {:.2f}".format(random_forest_precision_test))
print("Recall: {:.2f}".format(random_forest_recall_test))
print("F1 score: {:.2f}".format(random_forest_f1_test))
print()
print("Confusion matrix:\n", random_forest_confusion_test)
print()
print("Mean Squared Error: {:.2f}".format(random_forest_mse_test))
print("Mean Absolute Error: {:.2f}".format(random_forest_mae_test))
print("R2 score: {:.2f}".format(random_forest_r2_test))
print("Root Mean Squared Error: {:.2f}".format(random_forest_rmse_test))

list_of_random_forest_testing_stats = [random_forest_accuracy_test, random_forest_precision_test, random_forest_recall_test, random_forest_mse_test, random_forest_mae_test, random_forest_rmse_test, random_forest_r2_test]


In [ ]:
print("Classification Report:\n", classification_report(random_forest_y_train, random_forest_y_train_pred_class))

In [ ]:
metrics = ['Accuracy', 'Precision', 'Recall', 'MSE', 'MAE', 'RMSE', 'R2']

for i in range(len(metrics)):
    print(metrics[i])
    print('='*30)
    print('Trained Data: ', list_of_random_forest_training_stats[i])
    print('Tested Data: ', list_of_random_forest_testing_stats[i])
    print()

The performance of the random forest model between training and test data is, similar to the previous two models, quite high. Accuracy, recall and F1 score remain consistent, while precision is different by 0.02. Performance on the confusion matrices are, again, very similar too. So far,the random forst model seems to be the most accurate model by a very small margin, with a performance increase of .01 over the logistic regression model and .02 over the linear regression model.

## 4.4.2 K-Fold Cross Validation

In [ ]:
#performing 5 fold K-fold cross validation on random forest model using full dataset

#concatenating train and test datasets for independent and dependent variables
X = pd.concat([train[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']], 
                                 test[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]])
y = pd.concat([train['death_yn'], test['death_yn']]) 

#folds for cross validation
folds = 5

#creating K-Fold object
kf = KFold(n_splits=folds, shuffle=True, random_state=42)

#K-fold cross validation scores
mse_scores = -cross_val_score(rfc, X, y, cv=folds, scoring="neg_mean_squared_error")

#printing results
print(f"MSE scores: {mse_scores}")

Again ,the MSE scores produced by the K-Fold cross validation process performed on the randon forest model are quite low, with none exceeding .097. This indicates that the random forest implementation is relatively sucessful and predicting the dependent outcome based on the provided independent features, even when performed on subsets of the full dataset. This is consistent with the relatively high accuracy yielded by the model when performing analysis of the testing and training data. Taken together, the model performance information and the K-fold cross validation information give us a good indication that this model would likely perform well on new data. 

In terms of comparison to the other models, the random forest model performed slightly better on the cross validation than the logistic regression model, and slightly worse than the linear regression model. Due to the closeness in performance to the other models, and the higher overall accuracy of the random forest model, the best overall model for this predictive problem is likely debateable, with arguments for the suitability of each of the three.

# Part 5

## 5.1.1 Comparing the Performance of Each of our Models

In [ ]:
metrics = ['Accuracy', 'Precision', 'Recall', 'MSE', 'MAE', 'RMSE', 'R2']
models = ['Linear Regression', 'Logistic Regression', 'Random Forest']
num_models = len(models)

data = np.array([list_of_linear_regression_training_stats, 
                 list_of_logistic_regression_training_stats, 
                 list_of_random_forest_training_stats])

fig, axes = plt.subplots(nrows=len(metrics), ncols=1, figsize=(20, 6 * len(metrics)))

for i, metric in enumerate(metrics):
    ax = axes[i]
    for model in range(num_models):
        ax.bar(models[model], data[model, i], width=0.3, color=['#8FBBAF', '#C06C84', '#6C557B'][model])
    fig.set_facecolor('#f5f5f5')    
    ax.set_title(metric, fontsize=20)

plt.subplots_adjust(hspace=1)
plt.tight_layout()
plt.show()

In [ ]:
metrics = ['Accuracy', 'Precision', 'Recall', 'MSE', 'MAE', 'RMSE', 'R2']
models = ['Linear Regression', 'Logistic Regression', 'Random Forest']

data = np.array([list_of_linear_regression_training_stats, 
                 list_of_logistic_regression_training_stats, 
                 list_of_random_forest_training_stats])

for i, metric in enumerate(metrics):
    print(f"\n{metric}:")
    for j, model in enumerate(models):
        value = data[j, i]
        print(f"{model}: {value}")


## 5.2

### How to improve: 

paragraph on each of the below

1. Feature Selection
2. Creating New Features
3. Combining Predictive Models
4. Revised Splitting protocol
5. Feature Rescaling
6. Feature Ranking